In [1]:
import pandas as pd
import numpy as np
import os, sys, pysam, gzip, subprocess, shlex, time
from datetime import datetime
from tqdm import tqdm
from multiprocessing import Pool
from sklearn.preprocessing import OneHotEncoder

def give_inputs(cell_line):
    
    files_path = "/lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_datasets/{}".format(cells)
    
    if cell_line == "HEK293T":  
        samples = [["outTable_599710609", "outTable_905657585", "outTable_530905096"],
                   ["outTable_572868058", "outTable_364841872", "outTable_814257267"],
                   ["outTable_110067244", "outTable_597789462", "outTable_208420383"]]

        rmsk_file = "rmsk_hg38.sorted.gtf.gz"
        refseq_file = "hg38.110.ncbiRefSeq.sorted.gtf.gz"

    elif cell_line == "HEK":
        samples = [["outTable_724242056", "outTable_816573740"],
                   ["outTable_580067564", "outTable_718392497"],
                   ["outTable_181728208", "outTable_854894021"]]
        rmsk_file = "rmsk.sorted.gtf.gz"
        refseq_file = "hg19.ncbiRefSeq.sorted.gtf.gz"

    elif cell_line == "a549":
        samples = [["outTable_192318299", "outTable_436061877"],
                   ["outTable_535670354", "outTable_396704193"],
                   ["outTable_773331943", "outTable_302610513"]]
        rmsk_file = "rmsk.sorted.gtf.gz"
        refseq_file = "hg19.ncbiRefSeq.sorted.gtf.gz"

    else:
        samples= [["outTable_853538513", "outTable_921089530"]]
        rmsk_file = "rmsk.sorted.gtf.gz"
        refseq_file = "hg19.ncbiRefSeq.sorted.gtf.gz"
    
    return samples, rmsk_file, refseq_file, files_path

def extraction(prefix,  AG_min, AGfreq_threshold, cov_threshold, interval):    
    
    starttime = datetime.now()

    editing = []
                                         
    with gzip.open(prefix+".gz") as redi:
        for c,l in enumerate(redi):
            line = l.decode("utf-8").rstrip().split("\t")
            if line[0].find("chr") != -1:
                if line[2] == "A":
                    if line[4] != "-":
                        if int(line[4]) >= cov_threshold:
                            if "AG" in line[7]:    
                                AG_rna = eval(line[6])[2]/sum(eval(line[6]))
                                if AG_rna >= AGfreq_threshold:
                                    if eval(line[6])[2] >= AG_min:
                                        editing.append(line)

            if c % 50000000 == 0:
                print(f"\tSites evaluated: {c}")
    print("Total evaluated rows:", c)
    editing = pd.DataFrame(editing)
    stoptime = datetime.now()
    print(f"[{datetime.now()}] Extraction of Editing Candidates finished for current sample. Elapsed time: {stoptime-starttime}.")
    columns = ["Region", "Position", "Ref", "Strand", "Cov", "Qual", "Bases", "AllSubs", "Freq", "gCov", "gQual", "g[A,C,G,T]", "gAllSubs", "gFreq"]
    editing.columns = columns
    print(f"[{datetime.now()}] Starting extraction of intervals.")
    ohe = OneHotEncoder()
    ohe.fit(np.array(["A", "C", "G", "T"]).reshape(-1, 1))

    intervals = []
    starttime_preds = datetime.now()
    total_extracted = 0
    features_extracted_filepath = prefix+ "_feature_vectors.tsv"
    features_extracted = open(features_extracted_filepath, "w")

    df = editing.query("Region != 'chrM'")
    print(f"[{datetime.now()}] Loading reditable with tabix and pysam:", prefix)
    start_time = datetime.now()
    srr = pysam.TabixFile(prefix+".gz")
    with tqdm(total=df.shape[0], position=0, leave=True) as pbar:
        for site in df.itertuples():
            start = int(site.Position) - ((interval-1)/2)
            stop = int(site.Position) + ((interval-1)/2)
            AGrna = eval(site.Bases)[2]/sum(eval(site.Bases))
            srr_interval = []
            for s in srr.fetch(site.Region, start-1, stop):
                srr_interval.append(s.split("\t"))
            srr_interval = pd.DataFrame(srr_interval, columns=columns)
            if srr_interval.shape[0] == interval and len(set(srr_interval["Strand"])) == 1:
                intervals.append([site.Region, site.Position, site.Ref, site.Strand, AGrna, site.Bases, start, stop, stop-start + 1, srr_interval.shape[0]])
                total_extracted += 1
                strand = site.Strand
                seq = srr_interval.Ref.values.reshape(-1,1)
                seq_ohe = ohe.transform(seq).toarray().T
                vects_freqs = []
                strands = []
                vects = []
                for vect in srr_interval["Bases"]:
                    vect = np.array(eval(vect))
                    cov = sum(vect)
                    vect_freqs = vect / cov
                    vects_freqs.append(vect_freqs)
                    vects.append(vect)
                vects_freqs = np.array(vects_freqs).T
                vects = np.array(vects).T
                encoded_site = pd.concat([pd.DataFrame(seq_ohe), pd.DataFrame(vects_freqs)])
                encoded_site.reset_index(drop=True, inplace=True)
                if strand == 0: 
                    encoded_site = pd.DataFrame(np.flip(encoded_site.values, axis=1))
                encoded_site.to_csv(features_extracted, mode="a", sep="\t", header = None, index=None)
            pbar.update(1)
    intervals = pd.DataFrame(intervals)
    stop_time_global = datetime.now()
    print(f"[{datetime.now()}] Features Extraction Finished. Elapsed time {datetime.now()-starttime_preds}.")
    features_extracted.close()
    
    intervals.columns = ["Region", "Position", "RefBase", "Strand", "FreqAGrna", "BasesCounts", "Start", "Stop", "Intlen", "TabixLen"]
    intervals.to_csv(prefix + "_intervals.tsv", sep="\t", index=None)
    print(f"[{datetime.now()}] Computation Finished. Total Elapsed time: {datetime.now()-starttime}")

def candidates_bona_fide_extraction(name1, name2, path, cells, AG_min, AGfreq_threshold, cov_threshold, rna_cov_threshold):    

    starttime = datetime.now()
    sites = []
    
    wgs = pysam.TabixFile(f"{path}/{cells}_WGS.gz")   
    inactive = pysam.TabixFile(f"{path}/{name2}.gz")
    with gzip.open(f"{path}/{name1}.gz") as redi:
        for c,l in enumerate(redi):
            line = l.decode("utf-8").rstrip().split("\t")
            if line[0].find("chr") != -1:
                if line[0] != "chrM":
                    strand = int(line[3])
                    if strand != 2:
                        if line[2] == "A":
                            if line[4] != "-":
                                if int(line[4]) >= rna_cov_threshold:
                                    if line[7] == "AG":    
                                        AG_rna =  float(line[8])  
                                        if AG_rna >= AGfreq_threshold:
                                            if eval(line[6])[2] >= AG_min:
                                                region = line[0]
                                                start = int(line[1])-1
                                                stop = int(line[1])
                                                for ROW_inactive in inactive.fetch(region, start, stop):
                                                    row_inactive = ROW_inactive.split("\t")
                                                    if row_inactive[4] != "-":
                                                        if int(row_inactive[3]) == strand:
                                                            if int(row_inactive[4]) >= rna_cov_threshold:
                                                                sub_inactive = row_inactive[7]
                                                                for ROW_WGS in wgs.fetch(region, start, stop):
                                                                    row_wgs = ROW_WGS.split("\t")
                                                                    if row_wgs[9] !=  "-":
                                                                        if int(row_wgs[3]) == strand:
                                                                            if int(row_wgs[9])>=cov_threshold:
                                                                                sub_wgs = row_wgs[12]
                                                                                if sub_inactive == "-":
                                                                                    if sub_wgs == "-":
                                                                                        sites.append([line[0], line[1], 1])
                                                                                else:
                                                                                    if sub_inactive == "AG":
                                                                                        inactive_AG = eval(row_inactive[6])[2]
                                                                                        if inactive_AG >= AG_min:
                                                                                            inactive_freq = float(row_inactive[8]) 
                                                                                            if inactive_freq >= AGfreq_threshold:
                                                                                                if sub_wgs == "AG":
                                                                                                    sites.append([line[0], line[1], 0])
                                                                                                    
                                                                                                               
            if c % 50000000 == 0:
                print(f"\tSites evaluated: {c}")
     
    sites = pd.DataFrame(sites)
    sites.columns = ["Region", "Position", "Class"]
    positives = sites[sites.loc[:, "Class"]==1].shape[0]
    negatives = sites[sites.loc[:, "Class"]==0].shape[0]
    print("Total evaluated rows:", c)
    stoptime = datetime.now()
    
    print(f"[{datetime.now()}] Extraction of candidates bonafide sites finished for {name1} {name2} samples. Elapsed time: {stoptime-starttime}.")
    sites.to_csv(f"{path}/{name1}_{name2}_candidates_bona_fide_sites.tsv", sep="\t", index=None)
                                                                              
def bonafide_identification(path, rmsk, refseq):
                           
    u_path = "/lustrehome/pietrolucamazzacuva/filezilla-recas/scripts/utilities"                                                                            
                                                                                 
    for file_name in os.listdir(path):
        if file_name.find("_candidates_bona_fide_sites.tsv") !=-1:
            df = pd.read_csv(os.path.join(path, file_name), sep="\t")
            df.to_csv(os.path.join(path, file_name), sep="\t", index=None, header=False)
            name = file_name.replace(".tsv", "")
            cmd_sh = "python3 {}/AnnotateTablePython3.py -a {}/{} -n rmsk -i {}/{}.tsv -o {}/{}.out.rmsk -u".format(u_path, u_path, rmsk, path, name, path, name)
            args = shlex.split(cmd_sh)
            p = subprocess.Popen(args, env=dict(os.environ, PATH="/lustrehome/pietrolucamazzacuva/anaconda3/envs/tensorflow-gpu/bin"))

    time.sleep(300)

    for name in os.listdir(path):
        if name.find("_candidates_bona_fide_sites.out.rmsk") != -1:
            cmd_sh = "python3 {}/AnnotateTablePython3.py -a {}/{} -i {}/{} -o {}/{}.refseq -u".format(u_path, u_path, refseq, path, name, path, name)
            args = shlex.split(cmd_sh)
            p = subprocess.Popen(args, env=dict(os.environ, PATH="/lustrehome/pietrolucamazzacuva/anaconda3/envs/tensorflow-gpu/bin"))

    time.sleep(300)

    cols = ["Region", "Position", "Class", "RMSK-Rep", "RMSK-Reg", "RefSeq-Rep", "RefSeq-Reg"]
    for file_name in os.listdir(path):
        if file_name.find("_candidates_bona_fide_sites.out.rmsk.refseq") !=-1:
            df = pd.read_table(os.path.join(path, file_name), header=None)
            name = file_name.replace(".out.rmsk.refseq", "_annoted.tsv")
            df.columns = cols
            df.to_csv(os.path.join(path, name), sep="\t", index=None)

    for file_name in os.listdir(path):
        if file_name.find("candidates_bona_fide_sites_annoted.tsv") !=-1:
            bona_fide = pd.read_csv(os.path.join(path, file_name), sep="\t")
            rep = bona_fide[(bona_fide.iloc[:, 3] != "-") & (bona_fide.iloc[:, 4] != "-")]
            non_rep = bona_fide[(bona_fide.iloc[:, 3] == "-") & (bona_fide.iloc[:, 4] == "-")]

            del bona_fide

            non_rep_n = non_rep[non_rep.iloc[:, 2]==0]
            non_rep_p = non_rep[non_rep.iloc[:, 2]==1]

            del non_rep

            non_rep_p = non_rep_p[(non_rep_p.iloc[:, 5] != "-") & (non_rep_p.iloc[:, 6] != "-")]
            bona_fide = pd.concat([rep, non_rep_n, non_rep_p])

            del rep, non_rep_n, non_rep_p

            name = file_name.replace("candidates_bona_fide_sites_annoted.tsv", "bona_fide_sites.tsv")
            bona_fide = bona_fide.sort_values(["Region", "Position"])
            bona_fide.to_csv(os.path.join(path, name), sep="\t", index=None)

In [2]:
min_dna_cov = 10
min_rna_cov = 50
min_AG_rate = 0.01
min_G = 3
seq_lenght = 101

cells = "a549"

samples, rmsk_file_name, refseq_file_name, filespath = give_inputs(cells)

inputs = []
for i in range(len(samples)):
    for j in range(len(samples[i])):
        inputs.append([os.path.join(filespath, f"{samples[i][j]}"), min_G, min_AG_rate, min_rna_cov, seq_lenght])

with Pool(9) as pool:
    pool.starmap(extraction, inputs)

inputs = []
for i in range(3):
    inputs.append([samples[i][0], samples[i][1], filespath, cells, min_G, min_AG_rate, min_dna_cov, min_rna_cov])

with Pool(6) as pool:
    pool.starmap(candidates_bona_fide_extraction, inputs)     

bonafide_identification(filespath, rmsk_file_name, refseq_file_name)

	Sites evaluated: 0
	Sites evaluated: 0	Sites evaluated: 0	Sites evaluated: 0


	Sites evaluated: 0	Sites evaluated: 0

	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 200000000
	Sites evaluated: 200000000
	Sites evaluated: 200000000
	Sites evaluated: 200000000
	Sites evaluated: 200000000
	Sites evaluated: 200000000
Total evaluated rows: 216398797
[2024-08-12 07:59:07.329477] Extraction of Editing Candidates finished for current sample. Elapsed time: 0:04:30.169493.
[2024-08-12 07:59:07.331840] Starting extraction of intervals

100%|██████████| 6631/6631 [00:20<00:00, 323.44it/s]


[2024-08-12 07:59:27.886180] Features Extraction Finished. Elapsed time 0:00:20.549562.
[2024-08-12 07:59:27.910228] Computation Finished. Total Elapsed time: 0:04:50.750274
Total evaluated rows: 235536788
[2024-08-12 07:59:35.856280] Extraction of Editing Candidates finished for current sample. Elapsed time: 0:04:58.692904.
[2024-08-12 07:59:35.859263] Starting extraction of intervals.
[2024-08-12 07:59:35.883384] Loading reditable with tabix and pysam: /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_datasets/a549/outTable_535670354


  1%|          | 126/16870 [00:00<00:53, 311.29it/s]

Total evaluated rows: 236217106
[2024-08-12 07:59:36.417003] Extraction of Editing Candidates finished for current sample. Elapsed time: 0:04:59.252981.
[2024-08-12 07:59:36.419838] Starting extraction of intervals.
[2024-08-12 07:59:36.437291] Loading reditable with tabix and pysam: /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_datasets/a549/outTable_773331943


...

 79%|███████▉  | 15071/19055 [00:46<00:11, 340.24it/s]

[2024-08-12 08:00:27.095515] Features Extraction Finished. Elapsed time 0:00:51.232344.
[2024-08-12 08:00:27.138574] Computation Finished. Total Elapsed time: 0:05:49.975239


100%|██████████| 19055/19055 [00:58<00:00, 326.67it/s]


[2024-08-12 08:00:39.410338] Features Extraction Finished. Elapsed time 0:00:58.483835.
[2024-08-12 08:00:39.459621] Computation Finished. Total Elapsed time: 0:06:02.300138
	Sites evaluated: 0
	Sites evaluated: 0
	Sites evaluated: 0
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 200000000
	Sites evaluated: 200000000
	Sites evaluated: 200000000
Total evaluated rows: 235536788
[2024-08-12 08:07:23.556904] Extraction of candidates bonafide sites finished for outTable_535670354 outTable_396704193 samples. Elapsed time: 0:06:44.011600.
Total evaluated rows: 236217106
[2024-08-12 08:07:23.690745] Extraction of candidates bonafide sites finished for outTable_773331943 outTable_302610513 samples. Elapsed time: 0:06:44.145326.
Total evaluated rows: 241316303
[2024-08-12 08:07:4

Pysam version used: 0.15.4
Script time --> START: 12/08/2024 08:07:47
Table saved on /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_datasets/a549/outTable_192318299_outTable_436061877_candidates_bona_fide_sites.out.rmsk
Script time --> END: 12/08/2024 08:08:10
Pysam version used: 0.15.4
Script time --> START: 12/08/2024 08:07:47
Table saved on /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_datasets/a549/outTable_773331943_outTable_302610513_candidates_bona_fide_sites.out.rmsk
Script time --> END: 12/08/2024 08:08:10
Pysam version used: 0.15.4
Script time --> START: 12/08/2024 08:07:47
Table saved on /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_datasets/a549/outTable_535670354_outTable_396704193_candidates_bona_fide_sites.out.rmsk
Script time --> END: 12/08/2024 08:08:11
Pysam version used: 0.15.4
Script time --> START: 12/08/2024 08:09:44
Table saved on /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_d

In [4]:
min_dna_cov = 10
min_rna_cov = 30
min_AG_rate = 0.01
min_G = 2
seq_lenght = 101

cells = "HEK"

samples, rmsk_file_name, refseq_file_name, filespath = give_inputs(cells)
  
inputs = []
for i in range(len(samples)):
    for j in range(len(samples[i])):
        inputs.append([os.path.join(filespath, f"{samples[i][j]}"), min_G, min_AG_rate, min_rna_cov, seq_lenght])

with Pool(9) as pool:
    pool.starmap(extraction, inputs)
    
inputs = []
for i in range(3):
    inputs.append([samples[i][0], samples[i][1], filespath, cells, min_G, min_AG_rate, min_dna_cov, min_rna_cov])

with Pool(6) as pool:
    pool.starmap(candidates_bona_fide_extraction, inputs)     

bonafide_identification(filespath, rmsk_file_name, refseq_file_name)

	Sites evaluated: 0
	Sites evaluated: 0
	Sites evaluated: 0
	Sites evaluated: 0
	Sites evaluated: 0
	Sites evaluated: 0


Pysam version used: 0.15.4
Script time --> START: 12/08/2024 08:36:06
Table saved on /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_datasets/HEK293T/outTable_208420383_outTable_905657585_candidates_bona_fide_sites.out.rmsk.refseq
Script time --> END: 12/08/2024 08:38:09
Pysam version used: 0.15.4
Script time --> START: 12/08/2024 08:36:06
Table saved on /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_datasets/HEK293T/outTable_814257267_outTable_364841872_candidates_bona_fide_sites.out.rmsk.refseq
Script time --> END: 12/08/2024 08:38:18
Pysam version used: 0.15.4
Script time --> START: 12/08/2024 08:36:06
Table saved on /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_datasets/HEK293T/outTable_530905096_outTable_597789462_candidates_bona_fide_sites.out.rmsk.refseq
Script time --> END: 12/08/2024 08:38:27


	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 200000000
	Sites evaluated: 200000000
	Sites evaluated: 200000000
	Sites evaluated: 200000000
	Sites evaluated: 200000000
	Sites evaluated: 200000000
	Sites evaluated: 250000000
	Sites evaluated: 250000000
	Sites evaluated: 250000000
	Sites evaluated: 250000000
	Sites evaluated: 250000000
	Sites evaluated: 250000000
	Sites evaluated: 300000000
	Sites evaluated: 300000000
	Sites evaluated: 300000000
	Sites evaluated: 300000000
	Sites evaluated: 300000000
	Sites evaluated: 30000000

  7%|▋         | 375/5239 [00:05<00:53, 90.26it/s] 

Total evaluated rows: 422584480
[2024-08-12 08:46:46.515165] Extraction of Editing Candidates finished for current sample. Elapsed time: 0:08:38.820314.
[2024-08-12 08:46:46.517307] Starting extraction of intervals.
[2024-08-12 08:46:46.534328] Loading reditable with tabix and pysam: /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_datasets/HEK/outTable_181728208


...

100%|██████████| 12737/12737 [01:03<00:00, 201.07it/s]


[2024-08-12 08:49:04.257302] Features Extraction Finished. Elapsed time 0:01:03.457092.
[2024-08-12 08:49:04.290489] Computation Finished. Total Elapsed time: 0:10:56.595939


100%|██████████| 8460/8460 [01:00<00:00, 140.87it/s]


[2024-08-12 08:49:42.779057] Features Extraction Finished. Elapsed time 0:01:00.228432.
[2024-08-12 08:49:42.805519] Computation Finished. Total Elapsed time: 0:11:35.110546
	Sites evaluated: 0
	Sites evaluated: 0
	Sites evaluated: 0
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 200000000
	Sites evaluated: 200000000
	Sites evaluated: 200000000
	Sites evaluated: 250000000
	Sites evaluated: 250000000
	Sites evaluated: 250000000
	Sites evaluated: 300000000
	Sites evaluated: 300000000
	Sites evaluated: 300000000
	Sites evaluated: 350000000
	Sites evaluated: 350000000
	Sites evaluated: 350000000
	Sites evaluated: 400000000
	Sites evaluated: 400000000
	Sites evaluated: 400000000
	Sites evaluated: 450000000
Total evaluated rows: 422584480
[2024-08-12 09:01:42.197117] Extracti

Pysam version used: 0.15.4
Script time --> START: 12/08/2024 09:02:07
Table saved on /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_datasets/HEK/outTable_724242056_outTable_816573740_candidates_bona_fide_sites.out.rmsk
Script time --> END: 12/08/2024 09:02:26
Pysam version used: 0.15.4
Script time --> START: 12/08/2024 09:02:07
Table saved on /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_datasets/HEK/outTable_580067564_outTable_718392497_candidates_bona_fide_sites.out.rmsk
Script time --> END: 12/08/2024 09:02:30
Pysam version used: 0.15.4
Script time --> START: 12/08/2024 09:02:07
Table saved on /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_datasets/HEK/outTable_181728208_outTable_854894021_candidates_bona_fide_sites.out.rmsk
Script time --> END: 12/08/2024 09:02:36
Pysam version used: 0.15.4
Script time --> START: 12/08/2024 09:04:06
Table saved on /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_data

In [5]:
min_dna_cov = 10
min_rna_cov = 30
min_AG_rate = 0.01
min_G = 2
seq_lenght = 101

cells = "U87"

samples, rmsk_file_name, refseq_file_name, filespath = give_inputs(cells)
  
inputs = []
for i in range(len(samples)):
    for j in range(len(samples[i])):
        inputs.append([os.path.join(filespath, f"{samples[i][j]}"), min_G, min_AG_rate, min_rna_cov, seq_lenght])

with Pool(9) as pool:
    pool.starmap(extraction, inputs)
    
inputs = []
for i in range(1):
    inputs.append([samples[i][0], samples[i][1], filespath, cells, min_G, min_AG_rate, min_dna_cov, min_rna_cov])

with Pool(6) as pool:
    pool.starmap(candidates_bona_fide_extraction, inputs)     

bonafide_identification(filespath, rmsk_file_name, refseq_file_name)

	Sites evaluated: 0
	Sites evaluated: 0
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 200000000
	Sites evaluated: 200000000
Total evaluated rows: 213159783
[2024-08-12 09:10:30.350499] Extraction of Editing Candidates finished for current sample. Elapsed time: 0:04:23.735237.
[2024-08-12 09:10:30.353553] Starting extraction of intervals.
[2024-08-12 09:10:30.378151] Loading reditable with tabix and pysam: /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_datasets/U87/outTable_921089530


 51%|█████     | 4644/9180 [00:13<00:12, 356.94it/s]

Total evaluated rows: 225346133
[2024-08-12 09:10:44.128755] Extraction of Editing Candidates finished for current sample. Elapsed time: 0:04:37.514037.
[2024-08-12 09:10:44.130865] Starting extraction of intervals.
[2024-08-12 09:10:44.147168] Loading reditable with tabix and pysam: /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_datasets/U87/outTable_853538513


100%|██████████| 9180/9180 [00:26<00:00, 346.47it/s]]


[2024-08-12 09:10:57.013568] Features Extraction Finished. Elapsed time 0:00:26.655084.
[2024-08-12 09:10:57.041925] Computation Finished. Total Elapsed time: 0:04:50.426690


100%|██████████| 11663/11663 [00:34<00:00, 338.28it/s]


[2024-08-12 09:11:18.732449] Features Extraction Finished. Elapsed time 0:00:34.597876.
[2024-08-12 09:11:18.761879] Computation Finished. Total Elapsed time: 0:05:12.147182
	Sites evaluated: 0
	Sites evaluated: 50000000
	Sites evaluated: 100000000
	Sites evaluated: 150000000
	Sites evaluated: 200000000
Total evaluated rows: 225346133
[2024-08-12 09:17:20.854011] Extraction of candidates bonafide sites finished for outTable_853538513 outTable_921089530 samples. Elapsed time: 0:06:01.982123.


Pysam version used: 0.15.4
Script time --> START: 12/08/2024 09:17:22
Table saved on /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_datasets/U87/outTable_853538513_outTable_921089530_candidates_bona_fide_sites.out.rmsk
Script time --> END: 12/08/2024 09:17:42
Pysam version used: 0.15.4
Script time --> START: 12/08/2024 09:19:21
Table saved on /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_datasets/U87/outTable_853538513_outTable_921089530_candidates_bona_fide_sites.out.rmsk.refseq
Script time --> END: 12/08/2024 09:19:43


In [2]:
min_dna_cov = 10
min_rna_cov = 50
min_AG_rate = 0.01
min_G = 3
seq_lenght = 101

cells = "HEK293T"

samples, rmsk_file_name, refseq_file_name, filespath = give_inputs(cells)

inputs = []
for i in range(len(samples)):
    for j in range(len(samples[i])):
        inputs.append([os.path.join(filespath, f"{samples[i][j]}"), min_G, min_AG_rate, min_rna_cov, seq_lenght])

with Pool(9) as pool:
    pool.starmap(extraction, inputs)

inputs = []
for i in range(3):
    inputs.append([samples[i][0], samples[i][1], filespath, cells, min_G, min_AG_rate, min_dna_cov, min_rna_cov])
    inputs.append([samples[i][2], samples[i][1], filespath, cells, min_G, min_AG_rate, min_dna_cov, min_rna_cov])

with Pool(6) as pool:
    pool.starmap(candidates_bona_fide_extraction, inputs)     

bonafide_identification(filespath, rmsk_file_name, refseq_file_name)

	Sites evaluated: 0
	Sites evaluated: 0	Sites evaluated: 0
	Sites evaluated: 0	Sites evaluated: 0


	Sites evaluated: 0
	Sites evaluated: 0
	Sites evaluated: 0
	Sites evaluated: 0
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
Total evaluated rows: 194677901
[2024-08-12 14:58:25.243071] Extraction o

 19%|█▉        | 2330/12094 [00:06<00:27, 349.98it/s]

	Sites evaluated: 200000000


 23%|██▎       | 2783/12094 [00:08<00:28, 322.24it/s]

Total evaluated rows: 199831579
[2024-08-12 14:58:33.662586] Extraction of Editing Candidates finished for current sample. Elapsed time: 0:04:12.957696.
[2024-08-12 14:58:33.666338] Starting extraction of intervals.


...

 70%|███████   | 39904/56679 [02:20<00:57, 293.38it/s]

[2024-08-12 15:04:30.645221] Computation Finished. Total Elapsed time: 0:10:09.941982


100%|██████████| 56679/56679 [03:17<00:00, 286.88it/s]


[2024-08-12 15:05:28.085651] Features Extraction Finished. Elapsed time 0:03:17.713170.
[2024-08-12 15:05:28.219872] Computation Finished. Total Elapsed time: 0:11:07.520620
	Sites evaluated: 0
	Sites evaluated: 0	Sites evaluated: 0

	Sites evaluated: 0
	Sites evaluated: 0
	Sites evaluated: 0
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 50000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 100000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 150000000
	Sites evaluated: 200000000
	Sites evaluated: 200000000
Total evaluated rows: 199831579
[2024-08-12 15:11:40.602585] Extraction of candidates bonafide sites finished for outTable_110067244 outTable_597789462 

Pysam version used: 0.15.4
Script time --> START: 12/08/2024 15:16:34
Table saved on /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_datasets/HEK293T/outTable_110067244_outTable_597789462_candidates_bona_fide_sites.out.rmsk
Script time --> END: 12/08/2024 15:17:28
Pysam version used: 0.15.4
Script time --> START: 12/08/2024 15:16:34
Table saved on /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_datasets/HEK293T/outTable_572868058_outTable_364841872_candidates_bona_fide_sites.out.rmsk
Script time --> END: 12/08/2024 15:17:38
Pysam version used: 0.15.4
Script time --> START: 12/08/2024 15:16:34
Table saved on /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/independent_datasets/HEK293T/outTable_599710609_outTable_905657585_candidates_bona_fide_sites.out.rmsk
Script time --> END: 12/08/2024 15:17:54
Pysam version used: 0.15.4
Script time --> START: 12/08/2024 15:16:34
Table saved on /lustrehome/pietrolucamazzacuva/filezilla-recas/tissues/inde